#### 큰 흐름
- 라이브러리 로드
- 데이터 확인
- 모델 학습
- 제출 파일

---

##### 라이브러리 로드

In [13]:
# !pip install scikit-learn xgboost

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import mean_absolute_percentage_error


df = pd.read_csv("data/train.csv")
df



,신고접수번호,접수경로,신고접수일시,시군구,접수분류,긴급구조종류
0,JGT2IBW4,이동전화,20130101_0001,NaN,안내,NaN
1,74ED11Z4,기타,20130101_0002,NaN,안내,NaN
2,B4I8RIBW,이동전화,20130101_0002,NaN,안내,NaN
3,482FI3AJ,이동전화,20130101_0003,북구,출동,구급
4,AR9N3QT4,이동전화,20130101_0004,NaN,안내,NaN
...,...,...,...,...,...,...
6415797,KAOUMKLU,기타,20220630_2354,NaN,안내,NaN
6415798,LAI1MQ2K,이동전화,20220630_2354,사상구,출동,구급
6415799,N5KF2ELA,이동전화,20220630_2356,동구,출동,구급
6415800,53Q98D6Z,의료지도연결,20220630_2358,NaN,안내,NaN


In [16]:
# print(df["신고접수번호"].value_counts())
# print(df["접수경로"].value_counts())
# print(df["신고접수일시"].value_counts())
# print(df["시군구"].value_counts())
# print(df["접수분류"].value_counts().sum())
# print(df["긴급구조종류"].value_counts())
print(df["접수분류"].isnull().sum())

0


In [19]:
df["dt"] = df["신고접수일시"].apply(lambda x : datetime.strptime(x, "%Y%m%d_%H%M"))
df["_dt"] = df["dt"] + timedelta(hours=6)
df["_date"] = df._dt.dt.date.astype(str)
df["_hour"] = df._dt.dt.hour
df


,신고접수번호,접수경로,신고접수일시,시군구,접수분류,긴급구조종류,dt,_dt,_date,_hour
0,JGT2IBW4,이동전화,20130101_0001,NaN,안내,NaN,2013-01-01 00:01:00,2013-01-01 06:01:00,2013-01-01,6
1,74ED11Z4,기타,20130101_0002,NaN,안내,NaN,2013-01-01 00:02:00,2013-01-01 06:02:00,2013-01-01,6
2,B4I8RIBW,이동전화,20130101_0002,NaN,안내,NaN,2013-01-01 00:02:00,2013-01-01 06:02:00,2013-01-01,6
3,482FI3AJ,이동전화,20130101_0003,북구,출동,구급,2013-01-01 00:03:00,2013-01-01 06:03:00,2013-01-01,6
4,AR9N3QT4,이동전화,20130101_0004,NaN,안내,NaN,2013-01-01 00:04:00,2013-01-01 06:04:00,2013-01-01,6
...,...,...,...,...,...,...,...,...,...,...
6415797,KAOUMKLU,기타,20220630_2354,NaN,안내,NaN,2022-06-30 23:54:00,2022-07-01 05:54:00,2022-07-01,5
6415798,LAI1MQ2K,이동전화,20220630_2354,사상구,출동,구급,2022-06-30 23:54:00,2022-07-01 05:54:00,2022-07-01,5
6415799,N5KF2ELA,이동전화,20220630_2356,동구,출동,구급,2022-06-30 23:56:00,2022-07-01 05:56:00,2022-07-01,5
6415800,53Q98D6Z,의료지도연결,20220630_2358,NaN,안내,NaN,2022-06-30 23:58:00,2022-07-01 05:58:00,2022-07-01,5


In [105]:
df_ = df.pivot_table(index = ['_date', '_hour'], columns = '접수분류', aggfunc='size').reset_index()
print(df_)

접수분류        _date  _hour    안내    출동
0      2013-01-01      6  47.0  30.0
1      2013-01-01      7  55.0  26.0
2      2013-01-01      8  40.0  32.0
3      2013-01-01      9  24.0  21.0
4      2013-01-01     10  28.0  19.0
...           ...    ...   ...   ...
83227  2022-07-01      1  78.0  58.0
83228  2022-07-01      2  85.0  43.0
83229  2022-07-01      3  57.0  33.0
83230  2022-07-01      4  53.0  47.0
83231  2022-07-01      5  63.0  41.0

[83232 rows x 4 columns]


In [115]:
night_hour = df_[(df_["_hour"] >= 0) & (df_["_hour"] < 15)]
night_hour = night_hour[["_date", "안내", "출동"]].groupby("_date").sum()
night_hour = night_hour.reset_index()
night_hour.columns = ["date", "안내", "출동"]

day_hour = df_[(df_["_hour"] >= 15) & (df_["_hour"] < 24)]
day_hour = day_hour[["_date", "안내", "출동"]].groupby("_date").sum()
day_hour = day_hour.reset_index()
day_hour.columns = ["date", "안내", "출동"]

print(night_hour)
# print(day_hour)

print("\n==============\n")
night_date = df_[(df_["_hour"] >= 0) & (df_["_hour"] < 15)].groupby("_date").sum()
night_date["night_y"] = night_date["안내"] + night_date["출동"]
night_date = night_date.reset_index()
night_date.drop([ "안내", "출동"], axis=1, inplace=True)
# night_date.columns = ["date", "night_y"]

day_date = df_[(df_["_hour"] >= 15) & (df_["_hour"] < 24)].groupby("_date").sum()
day_date["day_y"] = day_date["안내"] + day_date["출동"]
day_date = day_date.reset_index()
day_date.drop([ "안내", "출동"], axis=1, inplace=True)
# day_date.columns = ["date", "day_y"]
# print(night_date)
# print(day_date)

# data = pd.concat([night, day], axis="columns")
# data = data.reset_index()
# # data
# data.columns = ["date", "night_y", "day_y", "안내", "출동"]
# # data
# drop_date = data.dropna()
# print(drop_date.shape)
# # check = df[["date", "y"]].groupby("date").count()
# # print(check)
# # drop_dates = list(check[check["y"]] <= 24)

            date     안내     출동
0     2013-01-01  406.0  207.0
1     2013-01-02  491.0  294.0
2     2013-01-03  766.0  247.0
3     2013-01-04  781.0  224.0
4     2013-01-05  467.0  324.0
...          ...    ...    ...
3464  2022-06-27  782.0  593.0
3465  2022-06-28  608.0  422.0
3466  2022-06-29  684.0  475.0
3467  2022-06-30  591.0  467.0
3468  2022-07-01  437.0  271.0

[3469 rows x 3 columns]




In [107]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# df_
# night_
# day_

night_dates = night_hour["date"].to_list()
night_0 = night_hour["안내"].to_list()
night_1 = night_hour["출동"].to_list()

day_dates = day_hour["date"].to_list()
day_0 = day_hour["안내"].to_list()
day_1 = day_hour["출동"].to_list()

fig = make_subplots(rows=2, cols=2)
fig.update_layout(title_text=f"119 안내 출동")

fig.add_trace(go.Scatter(x=night_dates, y=night_0, name="night_안내"), row=1, col=1)
fig.add_trace(go.Scatter(x=night_dates, y=night_1, name="night_출동"), row=2, col=1)
fig.add_trace(go.Scatter(x=day_dates, y=day_0, name="day_안내"), row=1, col=2)
fig.add_trace(go.Scatter(x=day_dates, y=day_1, name="day_출동"), row=2, col=2)
# fig.update_layout(
#     title='10 Year Time Series Data',
#     xaxis_title='Time',
#     yaxis_title='Value',
#     xaxis=dict(
#         tickformat='%Y-%m',  # x축의 형식을 년-월로 설정
#         title='Time',
#         tickmode='linear',   # x축 눈금을 균등하게 표시
#         dtick='M12'          # 눈금 간격을 12개월(1년)로 설정
#     )
# )

fig.show()
# 위에서 데이터 시각화 부분 copy
# data -> val 로 변경

# dates = val["date"].to_list()
# night_y = val["night_y"].to_list()
# day_y = val["day_y"].to_list()

# fig = make_subplots(rows=1, cols=1)
# fig.update_layout(title_text=f"Validation Data") # title 변경

# fig.add_trace(go.Scatter(x=dates, y=night_y, name="night_y"), row=1, col=1)
# fig.add_trace(go.Scatter(x=dates, y=day_y, name="day_y"), row=1, col=1)
# fig.show()

In [ ]:


# train, val = train_test_split(drop_date, train_size=0.8, shuffle=False)

# X_train = train[["date", "night_y"]]
# y_train = train[["date", "day_y"]]
# X_val = val[["date", "night_y"]]
# y_val = val[["date", "day_y"]]

# xgb_params = {
#     "n_estimator" : 100,
#     "max_depth" : 5,
#     "eval_metric" : "mape",
#     "early_stopping_rounds" : 10,
#     "random_state": 514
# }

# model = XGBRegressor(**xgb_params)
# eval_set = [(X_val['night_y'], y_val['day_y'])]

# model.fit(X=X_train['night_y'],
#           y=y_train['day_y'],
#           eval_set=eval_set,
#           verbose=True)
# prediction = model.predict(X_val['night_y'])

# mean_absolute_percentage_error(y_val['day_y'], prediction)

# test_df = pd.read_csv("data/test.csv")
# test_df["dt"] = test_df["신고접수일시"].apply(lambda x : datetime.strptime(x, "%Y%m%d_%H%M"))
# test_df["_dt"] = test_df["dt"] + timedelta(hours=6)
# test_df["_date"] = test_df._dt.dt.date.astype(str)
# test_df["_hour"] = test_df._dt.dt.hour
# test_df = test_df[["_date", "_hour", "신고접수번호"]].groupby(["_date", "_hour"]).count().reset_index()
# target_df = test_df[["_date", "신고접수번호"]].groupby("_date").sum()
# target_df = target_df.reset_index()
# target_df = target_df.rename(columns={"index": "date"})
# target_df.columns = ['date', 'night_y']
# # target_df

# prediction_result = model.predict(target_df['night_y'])

# submission = pd.read_csv('data/sample_submission.csv')
# submission["y"] = prediction_result.reshape(-1)
# submission.to_csv("submission.csv", index=False)

# import pickle

# # insert model_name
# model_name = "02" # <------
# ## Save pickle
# with open(f"{model_name}.pickle","wb") as f:
#     pickle.dump(model, f)

# # # Load pickle
# # with open(f"{model_name}.pickle","rb") as f:
# #     data = pickle.load(f)
# #     print(data)
